In [1]:
# Create IMAM supervision data to upload into RapidPro

# Identify all supervision (State and LGA) and put in rows by site (name, phone, email)
# Merge State and LGA supervision into complete listing of personnel
# export as .xlsx file

# Laurent said this is still very excel way of thinking. 
# We should just create the table in json and import to RapidPro

In [240]:
# To show plots in the notebook
%matplotlib inline  

import pandas as pd
import pandas_highcharts.core
from sqlalchemy import create_engine
import psycopg2
import matplotlib.pyplot 

import os

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "IMAM.settings")

from django.conf import settings
from home.management.commands.load_data import assign_state_lga_num, rename_cols, generic_cleaning, merge_in_and_outpatients, add_program_reports_from_supervision

import django
django.setup()

from home.models import First_admin, Second_admin, Site, Registration

In [266]:
engine = create_engine(
    'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{NAME}'.format(**settings.DATABASES['default']))
df = pd.read_sql_query("select * from registration;", con=engine)

In [242]:
# If there is no data in PostGres
# df = pd.ExcelFile('/home/robert/Downloads/reg.xlsx').parse('Contacts')

In [243]:
df.head()

,contact_uuid,urn,name,groups,siteid,type,first_seen,last_seen,post,mail
0,7be60a60-deb5-476d-8ac8-bcb7fd7c21a9,tel:+2348165573731,Mustapha Rawa.,,821110058,OTP,2017-04-28 05:22:08.603611,2017-04-28 06:53:36.968127,Community Health Officer,mustapharawa555@gmail.com
1,68e62c0f-1303-43fe-862a-0d1f6d17dae7,tel:+2347066919786,Salisu Adamu.,,1704110007,OTP,2016-09-05 11:08:09.302419,2017-02-06 09:10:31.742357,Community Health Officer,None
2,834c5cfb-c407-44ea-9dd9-e0a0c8710b5c,tel:+2349033285703,Abdullahi Rukaiyatu.,,221110009,OTP,2016-10-25 10:45:15.442123,2017-02-06 08:43:27.636584,Community Health Officer,antynorses70@gmail
3,87484df6-a292-4217-9a85-4f6960f01c41,tel:+2348038286150,Hassan Bukar.,,1704110007,OTP,2016-09-10 18:40:58.738249,2017-02-06 08:42:32.044403,Community Health Officer,None
4,683c7b28-a28e-4653-9fe0-6103b0caa08a,tel:+2348095578464,Lawan Hassan Bkudu .,,1703110007,OTP,2016-09-06 09:24:56.719761,2017-02-06 08:42:31.212486,In Charge Hospital/PHC,saleemlawan@yahoo.co.uk


In [244]:
# are any SiteIDs NaN ?
df.query('siteid!=siteid')

,contact_uuid,urn,name,groups,siteid,type,first_seen,last_seen,post,mail


In [245]:
df.query('siteid==1')

,contact_uuid,urn,name,groups,siteid,type,first_seen,last_seen,post,mail
973,e542626a-ee16-475c-9b23-710196fcec79,tel:+2348024753729,Robert Nokia.,,1,None,2016-08-24 18:37:03.116174,2017-04-24 08:33:58.546297,Coordinator,None
974,40bafd92-b832-48ad-b5b0-fc6ce6a25d60,tel:+2349074642972,Roberto Acer.,,1,None,2016-09-01 21:09:53.608496,2017-04-24 08:33:57.706249,Database Manager,None
975,e7de81db-f756-4d65-97a7-102e42f79712,tel:+2349074820020,Alessandro Ds Alex.,,1,None,2016-10-15 11:05:22.983034,2017-04-24 08:33:56.116195,Observer,None


In [246]:
# do any SiteIDs include the letter o instead of number zero? 
foo = pd.DataFrame({'a' : [1,2,3,4], 'b' : ['hi', 'ooo', 'fat', 'cat']})
foo[foo['b'].str.contains('ooo')]  

# Regex & Data Cleaning
# https://trendct.org/2016/08/05/real-world-data-cleanup-with-python-and-pandas/

,a,b
1,2,ooo


In [247]:
df[df['siteid'].str.contains('ooo')]
# can only use .str accessor with string
#.str.contains('\D')


AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [248]:
# To replace
# data['result'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [249]:
# run rename columns function
rename_cols(df)

,contact_uuid,urn,name,groups,siteid,type,first_seen,last_seen,post,mail
0,7be60a60-deb5-476d-8ac8-bcb7fd7c21a9,tel:+2348165573731,Mustapha Rawa.,,821110058,OTP,2017-04-28 05:22:08.603611,2017-04-28 06:53:36.968127,Community Health Officer,mustapharawa555@gmail.com
1,68e62c0f-1303-43fe-862a-0d1f6d17dae7,tel:+2347066919786,Salisu Adamu.,,1704110007,OTP,2016-09-05 11:08:09.302419,2017-02-06 09:10:31.742357,Community Health Officer,None
2,834c5cfb-c407-44ea-9dd9-e0a0c8710b5c,tel:+2349033285703,Abdullahi Rukaiyatu.,,221110009,OTP,2016-10-25 10:45:15.442123,2017-02-06 08:43:27.636584,Community Health Officer,antynorses70@gmail
3,87484df6-a292-4217-9a85-4f6960f01c41,tel:+2348038286150,Hassan Bukar.,,1704110007,OTP,2016-09-10 18:40:58.738249,2017-02-06 08:42:32.044403,Community Health Officer,None
4,683c7b28-a28e-4653-9fe0-6103b0caa08a,tel:+2348095578464,Lawan Hassan Bkudu .,,1703110007,OTP,2016-09-06 09:24:56.719761,2017-02-06 08:42:31.212486,In Charge Hospital/PHC,saleemlawan@yahoo.co.uk
5,e1847276-0ba9-472f-8870-f4f20284ebff,tel:+2348026838667,Abbaya Alhaji Wakil 1.,,807110017,OTP,2016-10-14 10:36:31.044905,2017-04-28 06:46:23.764083,In Charge Hospital/PHC,None
6,dc0732ec-7c05-4317-8d6c-b7fb4e026124,tel:+2348039687356,Talatu Ibrahim.,,1703110007,OTP,2016-09-06 10:49:38.792065,2017-02-06 08:42:31.106747,Community Health Officer,None
7,b2887928-d845-4ecd-acac-5e388fda0f6b,tel:+2347081323275,GARBA S GASI.,,804110010,OTP,2017-04-27 11:00:23.193493,2017-04-28 05:30:28.533177,None,None
8,2b585f2d-4f02-4753-8c02-fc3a2b43739f,tel:+2348080625085,Talatu mamman waba.,,813110007,OTP,2017-04-24 10:50:20.500276,2017-04-28 05:13:15.508710,Community Health Officer,talatumamman17@gmai.com
9,576c9310-c1ec-4b60-b380-727e47ebb70e,tel:+2347068789329,Sani Shehu.,,1702110018,OTP,2016-09-08 09:11:38.056070,2017-02-06 08:42:30.691347,Community Health Officer,None


In [250]:
# rename SiteID to siteid
# df=df.rename(columns = {'SiteID':'siteid'})

# Change the order (the index) of the columns
columnsTitles = ['siteid',                 
                 'name',
                 'urn',
                 'mail',
                 'post',
                 'type',
                 'groups',
                 'first_seen',
                 'last_seen'
                 ]

df2 = df.reindex(columns=columnsTitles)
df2

,siteid,name,urn,mail,post,type,groups,first_seen,last_seen
0,821110058,Mustapha Rawa.,tel:+2348165573731,mustapharawa555@gmail.com,Community Health Officer,OTP,,2017-04-28 05:22:08.603611,2017-04-28 06:53:36.968127
1,1704110007,Salisu Adamu.,tel:+2347066919786,None,Community Health Officer,OTP,,2016-09-05 11:08:09.302419,2017-02-06 09:10:31.742357
2,221110009,Abdullahi Rukaiyatu.,tel:+2349033285703,antynorses70@gmail,Community Health Officer,OTP,,2016-10-25 10:45:15.442123,2017-02-06 08:43:27.636584
3,1704110007,Hassan Bukar.,tel:+2348038286150,None,Community Health Officer,OTP,,2016-09-10 18:40:58.738249,2017-02-06 08:42:32.044403
4,1703110007,Lawan Hassan Bkudu .,tel:+2348095578464,saleemlawan@yahoo.co.uk,In Charge Hospital/PHC,OTP,,2016-09-06 09:24:56.719761,2017-02-06 08:42:31.212486
5,807110017,Abbaya Alhaji Wakil 1.,tel:+2348026838667,None,In Charge Hospital/PHC,OTP,,2016-10-14 10:36:31.044905,2017-04-28 06:46:23.764083
6,1703110007,Talatu Ibrahim.,tel:+2348039687356,None,Community Health Officer,OTP,,2016-09-06 10:49:38.792065,2017-02-06 08:42:31.106747
7,804110010,GARBA S GASI.,tel:+2347081323275,None,None,OTP,,2017-04-27 11:00:23.193493,2017-04-28 05:30:28.533177
8,813110007,Talatu mamman waba.,tel:+2348080625085,talatumamman17@gmai.com,Community Health Officer,OTP,,2017-04-24 10:50:20.500276,2017-04-28 05:13:15.508710
9,1702110018,Sani Shehu.,tel:+2347068789329,None,Community Health Officer,OTP,,2016-09-08 09:11:38.056070,2017-02-06 08:42:30.691347


In [264]:
df2 = assign_state_lga_num(df2)

In [252]:
# Create back-up
restore = df2

In [253]:
# Restore
df2 = restore

In [265]:
df2

,siteid,name,urn,mail,post,type,groups,first_seen,last_seen,siteid_lgt,state_num,lga_num
0,821110058,Mustapha Rawa.,tel:+2348165573731,mustapharawa555@gmail.com,Community Health Officer,OTP,,2017-04-28 05:22:08.603611,2017-04-28 06:53:36.968127,9,8,821
1,1704110007,Salisu Adamu.,tel:+2347066919786,None,Community Health Officer,OTP,,2016-09-05 11:08:09.302419,2017-02-06 09:10:31.742357,10,17,1704
2,221110009,Abdullahi Rukaiyatu.,tel:+2349033285703,antynorses70@gmail,Community Health Officer,OTP,,2016-10-25 10:45:15.442123,2017-02-06 08:43:27.636584,9,2,221
3,1704110007,Hassan Bukar.,tel:+2348038286150,None,Community Health Officer,OTP,,2016-09-10 18:40:58.738249,2017-02-06 08:42:32.044403,10,17,1704
4,1703110007,Lawan Hassan Bkudu .,tel:+2348095578464,saleemlawan@yahoo.co.uk,In Charge Hospital/PHC,OTP,,2016-09-06 09:24:56.719761,2017-02-06 08:42:31.212486,10,17,1703
5,807110017,Abbaya Alhaji Wakil 1.,tel:+2348026838667,None,In Charge Hospital/PHC,OTP,,2016-10-14 10:36:31.044905,2017-04-28 06:46:23.764083,9,8,807
6,1703110007,Talatu Ibrahim.,tel:+2348039687356,None,Community Health Officer,OTP,,2016-09-06 10:49:38.792065,2017-02-06 08:42:31.106747,10,17,1703
7,804110010,GARBA S GASI.,tel:+2347081323275,None,None,OTP,,2017-04-27 11:00:23.193493,2017-04-28 05:30:28.533177,9,8,804
8,813110007,Talatu mamman waba.,tel:+2348080625085,talatumamman17@gmai.com,Community Health Officer,OTP,,2017-04-24 10:50:20.500276,2017-04-28 05:13:15.508710,9,8,813
9,1702110018,Sani Shehu.,tel:+2347068789329,None,Community Health Officer,OTP,,2016-09-08 09:11:38.056070,2017-02-06 08:42:30.691347,10,17,1702


In [202]:
# What is the type of siteid
print type(df2['siteid'])
# This does not tell us anything about the variable

<class 'pandas.core.series.Series'>


In [203]:
df2['siteid'].describe()
# previously this would give the type of the var. 
# now referring to object

count    2.705000e+03
mean     1.695865e+09
std      1.255673e+09
min      2.000000e+00
25%      8.031100e+08
50%      1.807210e+09
75%      3.301110e+09
max      3.613210e+09
Name: siteid, dtype: float64

In [204]:
df2['siteid'] = pd.to_numeric(df2['siteid'], errors='coerce')

In [255]:
# Remove NaN from siteid

# If you don't specify a variable this will drop any rows including a NaN
df2 = df2.dropna(subset=['siteid'])

In [256]:
df2 = df2.query('siteid<3999990999')

In [257]:
df2 = df2.query('siteid>1')


In [258]:
df2 = df2.query('siteid!=99')

In [259]:
# This is supposed to remove non-numeric chars from siteid
# df2['siteid'] = filter(lambda x: x.isdigit(), df2['siteid'])

# for i in ('weeknum', 'state_num', 'lga_num', 'siteid'):
#     # IDs convert from string to float
#     dataframe[i] = pd.to_numeric(dataframe[i], errors='coerce')
#     # Clean out of range identification data - this deletes entire row where a NaN is found
#     dataframe = dataframe.query('%s==%s' % (i, i)).query('%s>=0' % i)
#     # Convert from float to int
#     dataframe[i] = dataframe[i].astype(int)


In [260]:
df2['siteid'] = df2['siteid'].astype(int)

In [261]:
df2['siteid'].describe()

count    2.705000e+03
mean     1.695865e+09
std      1.255673e+09
min      2.000000e+00
25%      8.031100e+08
50%      1.807210e+09
75%      3.301110e+09
max      3.613210e+09
Name: siteid, dtype: float64

In [160]:
# df2['siteid'].value_counts()

In [63]:
# Remove erroneous SiteIDs from df

#df2 = df2.query('1>siteid>3799990999')

#dataframe = dataframe.query('101110001<siteid<3799990999')

In [262]:
df2.state_num.value_counts()

8     460
35    453
21    333
20    323
2     278
17    212
33    178
36    142
19    117
18     65
5      52
16     50
81     12
80     11
82      7
3       2
51      2
22      2
1       2
50      1
43      1
4       1
78      1
Name: state_num, dtype: int64

In [213]:
df2.query('state_num==22')

,siteid,name,urn,mail,post,type,groups,first_seen,last_seen,siteid_lgt,state_num,lga_num


In [214]:
df2.lga_num.value_counts()

821     94
813     66
2106    60
816     43
3505    41
3514    39
3503    37
2011    37
2034    36
2004    35
805     34
3513    32
3510    31
3501    30
818     30
804     29
3509    28
3512    28
3517    28
2104    28
3602    27
3502    27
1703    25
3605    24
827     24
812     24
3508    23
3611    23
2010    22
3313    21
        ..
814      5
18       5
17       5
1706     4
1808     4
1807     4
1709     3
3316     3
1809     3
101      2
815      2
350      2
1713     2
2116     2
808      2
1801     2
2120     2
837      1
1929     1
823      1
3631     1
7812     1
443      1
2121     1
1802     1
1930     1
1700     1
1731     1
4310     1
1821     1
Name: lga_num, dtype: int64

In [263]:
#df2.query('lga_num==443')
# gives empty df

# Sort entire df by state_num or lga_num and review
df2.sort_values(by='state_num', ascending=0)

,siteid,name,urn,mail,post,type,groups,first_seen,last_seen,siteid_lgt,state_num,lga_num
280,825,Hadiza Mohammed Lawan.,tel:+2348064778550,None,Coordinator,None,,2016-10-15 11:11:20.205005,2017-04-27 16:44:26.105580,3,82,825
2129,824,Zara Modu.,tel:+2347068859699,None,Coordinator,None,,2016-10-15 11:09:00.625616,2017-01-18 20:03:19.173514,3,82,824
2130,821,Hadiza Shettima.,tel:+2348027411737,None,Coordinator,None,,2016-10-25 07:30:50.277345,2017-01-18 20:03:18.245247,3,82,821
971,824,Hadiza umar.,tel:+2347014165694,None,Coordinator,Sup,,2017-04-26 11:20:01.122108,2017-04-26 15:27:21.970127,3,82,824
2131,820,Yagana Mohammed.,tel:+2347035154445,yaganam6@gmail.com,Coordinator,None,,2016-10-15 11:04:10.704082,2017-01-18 20:03:17.339110,3,82,820
908,827,Jummai Sunday.,tel:+2348085380707,None,Coordinator,None,,2016-11-25 10:34:22.442976,2017-04-24 08:37:51.606678,3,82,827
862,820,Yagana Mohammed .,tel:+2347017378899,yaganam6@gmail.com,Coordinator,None,,2016-10-15 13:59:10.457992,2017-04-26 16:36:25.422307,3,82,820
281,818,Adama Salihu.,tel:+2348091841242,None,Coordinator,None,,2016-11-26 08:56:25.371501,2017-04-27 16:42:46.132408,3,81,818
935,813,MAAJI FALI.,tel:+2347019310856,None,Coordinator,Sup,,2017-04-26 11:21:00.571428,2017-04-26 16:34:30.291117,3,81,813
2700,816,Hadiza Achi.,tel:+2348029437389,None,Coordinator,None,,2016-10-16 16:03:15.047662,2017-01-18 20:03:13.794480,3,81,816


In [155]:
# create db with only supervision staff
supervision_df = df2[df2['siteid'] <= 3699]
# supervision siteids range from 1 to 3699

# Sort data
supervision_df = supervision_df.sort_values(by='siteid')
supervision_df

,siteid,name,urn,mail,post,type,groups,first_seen,last_seen,siteid_lgt,state_num,lga_num
1263,2,Hauwa Zoakah .,tel:+2348020938959,hauwabata@yahoo.com,Coordinator,None,,2016-10-26 09:04:19.462503,2017-04-24 08:35:35.817844,1,2,2
1264,2,Olawumi Monica Ajayi.,tel:+2348036173901,wumi.ajayi@yahoo.com,Technical Assistance,None,,2016-10-24 09:38:19.635591,2017-04-24 08:35:35.056843,1,2,2
1265,2,Wullanga Alfred,tel:+2347032853473,wulangaalfred@gmail.com,Coordinator,None,,2016-10-26 09:06:02.356386,2017-04-24 08:35:34.294050,1,2,2
1262,2,Ijagila Mark .,tel:+2348100001300,ijagilamark@gmail.com,Coordinator,None,,2016-10-24 09:36:56.209869,2017-04-24 08:35:36.598891,1,2,2
1149,2,Reuben Aidaticha.,tel:+2348088481006,aidaticha@gmail.com,Database Manager,None,,2016-10-24 10:38:14.041832,2017-04-24 08:35:37.132138,1,2,2
1148,2,Hauwa Zoakah.,tel:+2347035678763,hauwabata@yahoo.com,Coordinator,None,,2016-06-01 12:17:48.149140,2017-04-24 08:35:37.670118,1,2,2
1147,2,Reuben Aidaticha.,tel:+2348038213859,aidaticha@gmail.com,Database Manager,None,,2016-10-24 09:35:36.024432,2017-04-24 08:35:38.078705,1,2,2
1146,2,Wullanga Alfred.,tel:+2348029864318,wullangaalfred@gmail.com,Database Manager,None,,2016-10-24 09:35:46.229729,2017-04-24 08:35:38.481902,1,2,2
2174,5,Ali Shehu Kobi .,tel:+2347032683737,None,Stocks Manager,None,,2016-09-02 10:40:33.174405,2017-02-27 11:33:30.512228,1,5,5
2175,5,Habu Abdulmalik Dauda.,tel:+2348180448144,dhadboolagajei72@gmail.com,Database Manager,None,,2016-09-02 14:26:26.565091,2017-02-27 11:33:30.122641,1,5,5


In [156]:
# 13 states and 282 LGA = 294 total supervision sites
# Are there CMAM programs in all LGAs? 
supervision_df['siteid'].value_counts()

19      18
35      11
21      10
20      10
33       9
2        8
16       8
5        7
36       7
8        6
18       5
17       5
3512     5
3313     4
3503     4
3507     4
1823     4
1905     4
2011     4
2016     4
813      4
2106     4
2015     3
2018     3
3308     3
3306     3
202      3
3611     3
3304     3
3515     3
        ..
3514     1
2001     1
2007     1
2010     1
2024     1
2033     1
815      1
814      1
812      1
809      1
201      1
203      1
206      1
209      1
213      1
215      1
216      1
217      1
219      1
220      1
221      1
2034     1
3305     1
3319     1
3321     1
802      1
803      1
804      1
808      1
512      1
Name: siteid, dtype: int64

In [61]:
# How many registrations in supervision ? 
len(supervision_df['siteid'])

337

In [157]:
# Many supervision staff did not record their post - should recode them all as supervisors
supervision_df['post'].str.upper().value_counts()
# str.upper() does not make permanent change. 

# I think there was a data entry error with Post because most people entered their post correctly
# also there should not be any HC, HW in the supervision cadres

COORDINATOR             200
TECHNICAL ASSISTANCE     55
STOCKS MANAGER           34
DATABASE MANAGER         27
OBSERVER                  5
S                         2
D                         2
Name: post, dtype: int64

In [158]:
# to pivot data, create counts of each case by siteid

# df['count'] = df.groupby('col').cumcount() + 1
supervision_df['count'] = supervision_df.groupby('siteid').cumcount() + 1

In [159]:
supervision_df

,siteid,name,urn,mail,post,type,groups,first_seen,last_seen,siteid_lgt,state_num,lga_num,count
1263,2,Hauwa Zoakah .,tel:+2348020938959,hauwabata@yahoo.com,Coordinator,None,,2016-10-26 09:04:19.462503,2017-04-24 08:35:35.817844,1,2,2,1
1264,2,Olawumi Monica Ajayi.,tel:+2348036173901,wumi.ajayi@yahoo.com,Technical Assistance,None,,2016-10-24 09:38:19.635591,2017-04-24 08:35:35.056843,1,2,2,2
1265,2,Wullanga Alfred,tel:+2347032853473,wulangaalfred@gmail.com,Coordinator,None,,2016-10-26 09:06:02.356386,2017-04-24 08:35:34.294050,1,2,2,3
1262,2,Ijagila Mark .,tel:+2348100001300,ijagilamark@gmail.com,Coordinator,None,,2016-10-24 09:36:56.209869,2017-04-24 08:35:36.598891,1,2,2,4
1149,2,Reuben Aidaticha.,tel:+2348088481006,aidaticha@gmail.com,Database Manager,None,,2016-10-24 10:38:14.041832,2017-04-24 08:35:37.132138,1,2,2,5
1148,2,Hauwa Zoakah.,tel:+2347035678763,hauwabata@yahoo.com,Coordinator,None,,2016-06-01 12:17:48.149140,2017-04-24 08:35:37.670118,1,2,2,6
1147,2,Reuben Aidaticha.,tel:+2348038213859,aidaticha@gmail.com,Database Manager,None,,2016-10-24 09:35:36.024432,2017-04-24 08:35:38.078705,1,2,2,7
1146,2,Wullanga Alfred.,tel:+2348029864318,wullangaalfred@gmail.com,Database Manager,None,,2016-10-24 09:35:46.229729,2017-04-24 08:35:38.481902,1,2,2,8
2174,5,Ali Shehu Kobi .,tel:+2347032683737,None,Stocks Manager,None,,2016-09-02 10:40:33.174405,2017-02-27 11:33:30.512228,1,5,5,1
2175,5,Habu Abdulmalik Dauda.,tel:+2348180448144,dhadboolagajei72@gmail.com,Database Manager,None,,2016-09-02 14:26:26.565091,2017-02-27 11:33:30.122641,1,5,5,2


In [152]:
# Three data points to include in IMAM Supervision database for each supervision SiteID
# Name, Phone Number (URN), email

columnsTitles = ['siteid',                 
                 'name',
                 'urn',
                 'mail',
                 'state_num',
                 'lga_num',
                 'count',
                 ]

supervision_df = supervision_df.reindex(columns=columnsTitles)

In [153]:
# create db with only State Level supervision staff
state_df = supervision_df[supervision_df['siteid'] <= 39]


In [154]:
state_df

,siteid,name,urn,mail,state_num,lga_num,count
1263,2,Hauwa Zoakah .,tel:+2348020938959,hauwabata@yahoo.com,NaN,NaN,1
1264,2,Olawumi Monica Ajayi.,tel:+2348036173901,wumi.ajayi@yahoo.com,NaN,NaN,2
1265,2,Wullanga Alfred,tel:+2347032853473,wulangaalfred@gmail.com,NaN,NaN,3
1262,2,Ijagila Mark .,tel:+2348100001300,ijagilamark@gmail.com,NaN,NaN,4
1149,2,Reuben Aidaticha.,tel:+2348088481006,aidaticha@gmail.com,NaN,NaN,5
1148,2,Hauwa Zoakah.,tel:+2347035678763,hauwabata@yahoo.com,NaN,NaN,6
1147,2,Reuben Aidaticha.,tel:+2348038213859,aidaticha@gmail.com,NaN,NaN,7
1146,2,Wullanga Alfred.,tel:+2348029864318,wullangaalfred@gmail.com,NaN,NaN,8
2174,5,Ali Shehu Kobi .,tel:+2347032683737,None,NaN,NaN,1
2175,5,Habu Abdulmalik Dauda.,tel:+2348180448144,dhadboolagajei72@gmail.com,NaN,NaN,2


In [104]:
# convert vertical to horizontal database. 
# will None in the cells cause us to send excessive number of warning SMS? 
state_wide = state_df.pivot(index='siteid', columns='count')

In [105]:
state_wide.head()

name                          \
count                   1                       2    
siteid                                               
2           Hauwa Zoakah .   Olawumi Monica Ajayi.   
5         Ali Shehu Kobi .  Habu Abdulmalik Dauda.   
8            MAGRET AYUBA.  Abdullahi Alhaji Madi.   
16          Rukayya Lawal.       Selamawit Negash.   
17      Olatomiwa Olabisi.       Saidu Umar Adamu.   

                                                                   \
count                               3                          4    
siteid                                                              
2                      Wullanga Alfred             Ijagila Mark .   
5                Jackson Ladu Martins.               Yakubu Baba.   
8       Amarachi, Clementina Chukwuma.   Hassana Suleiman Jibrin.   
16                 Ahmed Audu Saddana.           Suleiman Mamman.   
17              Temidayo Esther Ajala.  MUSA  MOHAMMED  HADEJIA .   

                                                          \
count                         5                       6    
siteid                                                     
2              Reuben Aidaticha.           Hauwa Zoakah.   
5       Charity Evans Nysalamke.  Sama'ila Usman Maikan.   
8                 Daniel James .    Aminu Usman Danzomo.   
16                 Usman Baraya.      Ronas Amos Amusa .   
17        Shuaibu Aliyu Ringim .                    None   

                                                                         ...   \
count                               7                    8     9     10  ...    
siteid                                                                   ...    
2                    Reuben Aidaticha.     Wullanga Alfred.  None  None  ...    
5                  Hamza Yakubu Sade .                 None  None  None  ...    
8                                 None                 None  None  None  ...    
16      Olufunmilayo Adepoju-adebambo.  Ibrahim Inuwa Lano.  None  None  ...    
17                                None                 None  None  None  ...    

        mail                                                        
count     9     10    11    12    13    14    15    16    17    18  
siteid                                                              
2       None  None  None  None  None  None  None  None  None  None  
5       None  None  None  None  None  None  None  None  None  None  
8       None  None  None  None  None  None  None  None  None  None  
16      None  None  None  None  None  None  None  None  None  None  
17      None  None  None  None  None  None  None  None  None  None  

[5 rows x 54 columns]

In [106]:
# Create new column name
# and correct multiIndex
state_wide.columns = ["State" + (state_wide.columns[i][0]) + str(state_wide.columns[i][1]) for i in range(len(state_wide.columns))]

In [109]:
state = state_wide.reset_index()
state

,siteid,Statename1,Statename2,Statename3,Statename4,Statename5,Statename6,Statename7,Statename8,Statename9,...,Statemail9,Statemail10,Statemail11,Statemail12,Statemail13,Statemail14,Statemail15,Statemail16,Statemail17,Statemail18
0,2,Hauwa Zoakah .,Olawumi Monica Ajayi.,Wullanga Alfred,Ijagila Mark .,Reuben Aidaticha.,Hauwa Zoakah.,Reuben Aidaticha.,Wullanga Alfred.,None,...,None,None,None,None,None,None,None,None,None,None
1,5,Ali Shehu Kobi .,Habu Abdulmalik Dauda.,Jackson Ladu Martins.,Yakubu Baba.,Charity Evans Nysalamke.,Sama'ila Usman Maikan.,Hamza Yakubu Sade .,None,None,...,None,None,None,None,None,None,None,None,None,None
2,8,MAGRET AYUBA.,Abdullahi Alhaji Madi.,"Amarachi, Clementina Chukwuma.",Hassana Suleiman Jibrin.,Daniel James .,Aminu Usman Danzomo.,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,16,Rukayya Lawal.,Selamawit Negash.,Ahmed Audu Saddana.,Suleiman Mamman.,Usman Baraya.,Ronas Amos Amusa .,Olufunmilayo Adepoju-adebambo.,Ibrahim Inuwa Lano.,None,...,None,None,None,None,None,None,None,None,None,None
4,17,Olatomiwa Olabisi.,Saidu Umar Adamu.,Temidayo Esther Ajala.,MUSA MOHAMMED HADEJIA .,Shuaibu Aliyu Ringim .,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,18,Hauwa Usman .,Florence Ebun Oni.,Saratu Aduwak.,Jane Gwani.,Maryam Yusuf .,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,19,Auwalu Ibrahim.,Fumen Fuset Emmanuel.,Akilu Sani.,Sabo Wada.,"Abolarin, Samuel Sesan.",Sabo Wada.,Abigail Ishaya Nyam .,Deborah Abi Nyako.,Dorcas Heinmen Gauji.,...,dgauji@yahoo.com,ayodejiosunkentan@yahoo.com,murtalamuhd33@gmail.com,lekrunmon@yahoo.com,adomustaphabichi1@gmail.com,adosanda@gmail.com,akilumfashi@gmail.com,williamnyako22@gmail.com,msaniabdullahi@gmail.com,mikekush70@yahoo.com
7,20,Yusufahmadu Gangara.,Sim Karla Sakyenu.,Musbahu Hamisu.,Rabia Mohammed Sno .,Zayyana Isyaku Sule.,Ado Ibrahim Abdulrahman.,Abdulmumin Lawan.,Ibrahim Maje Sayyadi.,Hamisu Idris K/bai.,...,hamisuhara@yahoo.com,hadabdul@yahoo.co.uk,None,None,None,None,None,None,None,None
8,21,Aliyu Galadima Libata.,Shamsu Muhammed.,Abdulmalik Muhammad Illo.,Beatrice Kwere.,Abdulmalik Muhammad Illo .,Suwaiba M Bello.,Sunday Norbert.,Aliyu Galadima Libata .,Abimbola Abosede Sobo.,...,soboabimbola@yahoo.com,bisolaatoyebi@yahoo.com,None,None,None,None,None,None,None,None
9,30,30.,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [110]:
# Change None in cells to blank
for cell in state:
   state[cell].fillna(value='', inplace=True)

In [111]:
state

,siteid,Statename1,Statename2,Statename3,Statename4,Statename5,Statename6,Statename7,Statename8,Statename9,...,Statemail9,Statemail10,Statemail11,Statemail12,Statemail13,Statemail14,Statemail15,Statemail16,Statemail17,Statemail18
0,2,Hauwa Zoakah .,Olawumi Monica Ajayi.,Wullanga Alfred,Ijagila Mark .,Reuben Aidaticha.,Hauwa Zoakah.,Reuben Aidaticha.,Wullanga Alfred.,,...,,,,,,,,,,
1,5,Ali Shehu Kobi .,Habu Abdulmalik Dauda.,Jackson Ladu Martins.,Yakubu Baba.,Charity Evans Nysalamke.,Sama'ila Usman Maikan.,Hamza Yakubu Sade .,,,...,,,,,,,,,,
2,8,MAGRET AYUBA.,Abdullahi Alhaji Madi.,"Amarachi, Clementina Chukwuma.",Hassana Suleiman Jibrin.,Daniel James .,Aminu Usman Danzomo.,,,,...,,,,,,,,,,
3,16,Rukayya Lawal.,Selamawit Negash.,Ahmed Audu Saddana.,Suleiman Mamman.,Usman Baraya.,Ronas Amos Amusa .,Olufunmilayo Adepoju-adebambo.,Ibrahim Inuwa Lano.,,...,,,,,,,,,,
4,17,Olatomiwa Olabisi.,Saidu Umar Adamu.,Temidayo Esther Ajala.,MUSA MOHAMMED HADEJIA .,Shuaibu Aliyu Ringim .,,,,,...,,,,,,,,,,
5,18,Hauwa Usman .,Florence Ebun Oni.,Saratu Aduwak.,Jane Gwani.,Maryam Yusuf .,,,,,...,,,,,,,,,,
6,19,Auwalu Ibrahim.,Fumen Fuset Emmanuel.,Akilu Sani.,Sabo Wada.,"Abolarin, Samuel Sesan.",Sabo Wada.,Abigail Ishaya Nyam .,Deborah Abi Nyako.,Dorcas Heinmen Gauji.,...,dgauji@yahoo.com,ayodejiosunkentan@yahoo.com,murtalamuhd33@gmail.com,lekrunmon@yahoo.com,adomustaphabichi1@gmail.com,adosanda@gmail.com,akilumfashi@gmail.com,williamnyako22@gmail.com,msaniabdullahi@gmail.com,mikekush70@yahoo.com
7,20,Yusufahmadu Gangara.,Sim Karla Sakyenu.,Musbahu Hamisu.,Rabia Mohammed Sno .,Zayyana Isyaku Sule.,Ado Ibrahim Abdulrahman.,Abdulmumin Lawan.,Ibrahim Maje Sayyadi.,Hamisu Idris K/bai.,...,hamisuhara@yahoo.com,hadabdul@yahoo.co.uk,,,,,,,,
8,21,Aliyu Galadima Libata.,Shamsu Muhammed.,Abdulmalik Muhammad Illo.,Beatrice Kwere.,Abdulmalik Muhammad Illo .,Suwaiba M Bello.,Sunday Norbert.,Aliyu Galadima Libata .,Abimbola Abosede Sobo.,...,soboabimbola@yahoo.com,bisolaatoyebi@yahoo.com,,,,,,,,
9,30,30.,,,,,,,,,...,,,,,,,,,,


In [122]:
# Create same list for LGA
# create db with only LGA Level supervision staff
lga_df = supervision_df[supervision_df.siteid >= 101]
lga_df = lga_df[lga_df['siteid'] <= 3799]


In [123]:
lga_df

,siteid,name,urn,mail,count
1944,201,Edina Richard.,tel:+2348131816143,None,1
279,202,Monday Leasado.,tel:+2348086625852,None,1
201,202,Monday Leasado.,tel:+2348065356507,None,2
278,202,Monday Leasado.,tel:+2348038056048,None,3
175,203,Naaticha Waziri .,tel:+2348037675670,naatiwaziri69@gmail.com,1
1066,204,Rose Zidon .,tel:+2347065739356,None,1
1093,204,Rose Mbamuno Zidon,tel:+2349086559594,None,2
1085,205,Esther Danjuma.,tel:+2347030635580,None,1
1428,205,Esther Danjuma.,tel:+2348025568649,None,2
1084,206,Hannatu B Usman.,tel:+2348064811859,hannatubbu@gmail.com,1


In [131]:
# convert vertical to horizontal database. 
# will None in the cells cause us to send excessive number of warning SMS? 
lga_wide = lga_df.pivot(index='siteid', columns='count')

In [132]:
lga_wide

name                               \
count                            1                            2   
siteid                                                            
201                 Edina Richard.                         None   
202                Monday Leasado.              Monday Leasado.   
203              Naaticha Waziri .                         None   
204                   Rose Zidon .           Rose Mbamuno Zidon   
205                Esther Danjuma.              Esther Danjuma.   
206               Hannatu B Usman.                         None   
207               Aishatu Samaila.             Aishatu Samaila.   
208              Wubbewasu David .             Apollos Elkanah.   
209        Patience Zomti Douglas.                         None   
210             Ularamai Raphael .              Adama Abubakar.   
211               Yakubu Suleiman.             Yakubu Suleiman.   
212                Wilson Hosanna.             Aishatu Yuguda .   
213           Barnabas Paul Mbiya.                         None   
214                 Hadiza Gidado.               Hadiza Gidado.   
215              Grace John 72221.                         None   
216                 Pwaveno Eliel.                         None   
217              Hannatu I Koliro.                         None   
218             Blessing Ibrahim .            Blessing Ibrahim.   
219                  Shatu Daniel.                         None   
220           Nacha Bitrus Maksha.                         None   
221              Yuguda Abdullahi.                         None   
504             Ahmed Idi Dagauda.                         None   
512              Ibrahim Nalafiya.                         None   
513             Abdullahi M Bello.                         None   
802                Rebecca Thliza.                         None   
803                Abba Kawu Goni.                         None   
804                   Hauwa Aliyu.                         None   
805              Aishatu Abubakar.            Aishatu Abubakar.   
806                   Pana Tapchi.                           4.   
807                 Aisha Allamin.               Aisha Allamin.   
...                            ...                          ...   
3306              Abdullahi Usman.         Aminu Namakka Balle.   
3308             Bashiru Abubakar.            Bashiru Abubakar.   
3313                   Amina Bello                 Amina Bello.   
3317                 Asabe Shehu .                Asabe Shehu .   
3319            Mukhtar Abdullahi.                         None   
3321                 Rukayya Musa.                         None   
3501    Fatsuma Mustapha Mahammad.                         None   
3502                  Zanna Kasim.                         None   
3503           Umar Umar Muhammad.                Aisha Ahmed .   
3504                  Aisha a ali.                         None   
3505                  Asabe Dauda.                         None   
3506                 Naima Usman .                         None   
3507                Baba Isa Modu.                 Gambo Barde.   
3508                 Amina Yakubu.              Ali M Abubakar.   
3509               ADAMA ALHASSAN.                         None   
3510            Hadiza Maina Adam.           Hadiza Maina Adam.   
3511          Ngabarama kachallah.                         None   
3512          Yakubu Yusuf Degubi.     Jennifer, Ooja Inalegwu.   
3513                Hadiza idriss.               Hadiza idriss.   
3514          HAUWA IBRAHIM CHANA.                         None   
3515              Emmanuel Samuel.     Ngozi Odichinma Omenazu.   
3516           IBRAHIM MAI KASUWA.  Abdullahi Mohammed Ibrahim.   
3517         Bukar Sadiq mohammed.        Bukar Sadiq Mohammed.   
3602                  Usman Bello.               Hassan Hakimi.   
3603          Abdullahi B Suleman.                         None   
3605                Haliru Hassan.                  Rabiu Umar.   
3607      

In [133]:
# Rename columns and correct multiIndex
lga_wide.columns = ["LGA" + (lga_wide.columns[i][0]) + str(lga_wide.columns[i][1]) for i in range(len(lga_wide.columns))]

In [134]:
lga = lga_wide.reset_index()
lga

,siteid,LGAname1,LGAname2,LGAname3,LGAname4,LGAname5,LGAurn1,LGAurn2,LGAurn3,LGAurn4,LGAurn5,LGAmail1,LGAmail2,LGAmail3,LGAmail4,LGAmail5
0,201,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None
1,202,Monday Leasado.,Monday Leasado.,Monday Leasado.,None,None,tel:+2348086625852,tel:+2348065356507,tel:+2348038056048,None,None,None,None,None,None,None
2,203,Naaticha Waziri .,None,None,None,None,tel:+2348037675670,None,None,None,None,naatiwaziri69@gmail.com,None,None,None,None
3,204,Rose Zidon .,Rose Mbamuno Zidon,None,None,None,tel:+2347065739356,tel:+2349086559594,None,None,None,None,None,None,None,None
4,205,Esther Danjuma.,Esther Danjuma.,None,None,None,tel:+2347030635580,tel:+2348025568649,None,None,None,None,None,None,None,None
5,206,Hannatu B Usman.,None,None,None,None,tel:+2348064811859,None,None,None,None,hannatubbu@gmail.com,None,None,None,None
6,207,Aishatu Samaila.,Aishatu Samaila.,None,None,None,tel:+2348022645708,tel:+2348034154949,None,None,None,None,None,None,None,None
7,208,Wubbewasu David .,Apollos Elkanah.,None,None,None,tel:+2348174118980,tel:+2348029839844,None,None,None,None,apollospola@gmail,None,None,None
8,209,Patience Zomti Douglas.,None,None,None,None,tel:+2348065754427,None,None,None,None,None,None,None,None,None
9,210,Ularamai Raphael .,Adama Abubakar.,None,None,None,tel:+2347087576976,tel:+2349078543500,None,None,None,None,None,None,None,None


In [135]:
# Change None in cells to blank
for cell in lga:
   lga[cell].fillna(value='', inplace=True)

In [136]:
lga


,siteid,LGAname1,LGAname2,LGAname3,LGAname4,LGAname5,LGAurn1,LGAurn2,LGAurn3,LGAurn4,LGAurn5,LGAmail1,LGAmail2,LGAmail3,LGAmail4,LGAmail5
0,201,Edina Richard.,,,,,tel:+2348131816143,,,,,,,,,
1,202,Monday Leasado.,Monday Leasado.,Monday Leasado.,,,tel:+2348086625852,tel:+2348065356507,tel:+2348038056048,,,,,,,
2,203,Naaticha Waziri .,,,,,tel:+2348037675670,,,,,naatiwaziri69@gmail.com,,,,
3,204,Rose Zidon .,Rose Mbamuno Zidon,,,,tel:+2347065739356,tel:+2349086559594,,,,,,,,
4,205,Esther Danjuma.,Esther Danjuma.,,,,tel:+2347030635580,tel:+2348025568649,,,,,,,,
5,206,Hannatu B Usman.,,,,,tel:+2348064811859,,,,,hannatubbu@gmail.com,,,,
6,207,Aishatu Samaila.,Aishatu Samaila.,,,,tel:+2348022645708,tel:+2348034154949,,,,,,,,
7,208,Wubbewasu David .,Apollos Elkanah.,,,,tel:+2348174118980,tel:+2348029839844,,,,,apollospola@gmail,,,
8,209,Patience Zomti Douglas.,,,,,tel:+2348065754427,,,,,,,,,
9,210,Ularamai Raphael .,Adama Abubakar.,,,,tel:+2347087576976,tel:+2349078543500,,,,,,,,


In [ ]:
# merge state df to lga df
# Merge with left join
pd.merge(lga, state, on='state_num', how='left')

In [ ]:
# merge to implementation staff df

In [112]:
#convert to xls
# and/or export as JSON


In [ ]:
# loop over multiIndex names of columns
[x for x in state_wide.columns]

In [73]:
# This code did not work
df = state_wide
df.columns = [''.join(col).strip() for col in df.columns.values]

In [170]:
# This code did not work

# Rename column headers - multiIndex
# need to have only one line with column names at top of df

level_o = state_wide.columns.tolist()
columnsTitles =[]
for item in level_o:
    columnsTitles.append((item[0],"State"+item[0].capitalize()+str(item[1])))

print columnsTitles

[('name', 'StateName1'), ('name', 'StateName2'), ('name', 'StateName3'), ('name', 'StateName4'), ('name', 'StateName5'), ('name', 'StateName6'), ('name', 'StateName7'), ('name', 'StateName8'), ('name', 'StateName9'), ('name', 'StateName10'), ('name', 'StateName11'), ('name', 'StateName12'), ('name', 'StateName13'), ('name', 'StateName14'), ('name', 'StateName15'), ('name', 'StateName16'), ('name', 'StateName17'), ('name', 'StateName18'), ('urn', 'StateUrn1'), ('urn', 'StateUrn2'), ('urn', 'StateUrn3'), ('urn', 'StateUrn4'), ('urn', 'StateUrn5'), ('urn', 'StateUrn6'), ('urn', 'StateUrn7'), ('urn', 'StateUrn8'), ('urn', 'StateUrn9'), ('urn', 'StateUrn10'), ('urn', 'StateUrn11'), ('urn', 'StateUrn12'), ('urn', 'StateUrn13'), ('urn', 'StateUrn14'), ('urn', 'StateUrn15'), ('urn', 'StateUrn16'), ('urn', 'StateUrn17'), ('urn', 'StateUrn18'), ('mail', 'StateMail1'), ('mail', 'StateMail2'), ('mail', 'StateMail3'), ('mail', 'StateMail4'), ('mail', 'StateMail5'), ('mail', 'StateMail6'), ('mail', 

In [175]:
#Rename multiIndex

# want to have variable names with 
# - Level = State
# - Variable type = Name
# - Running number = 1 - 20
# Normally there are only 3 or 4 persons in supervision cadre at state level

corrected_varnames = state_wide.reindex(columns=columnsTitles)
# maybe should only rename columns not reindex here. 
# Use MultiIndex.rename ?

corrected_varnames
# why are all data are deleted ?
# http://stackoverflow.com/questions/34439437/how-to-reindex-a-a-pandas-dataframe-by-date-range-without-deleting-values

name                                                         \
count  StateName1 StateName2 StateName3 StateName4 StateName5 StateName6   
siteid                                                                     
2             NaN        NaN        NaN        NaN        NaN        NaN   
5             NaN        NaN        NaN        NaN        NaN        NaN   
8             NaN        NaN        NaN        NaN        NaN        NaN   
16            NaN        NaN        NaN        NaN        NaN        NaN   
17            NaN        NaN        NaN        NaN        NaN        NaN   
18            NaN        NaN        NaN        NaN        NaN        NaN   
19            NaN        NaN        NaN        NaN        NaN        NaN   
20            NaN        NaN        NaN        NaN        NaN        NaN   
21            NaN        NaN        NaN        NaN        NaN        NaN   
30            NaN        NaN        NaN        NaN        NaN        NaN   
33            NaN        NaN        NaN        NaN        NaN        NaN   
35            NaN        NaN        NaN        NaN        NaN        NaN   
36            NaN        NaN        NaN        NaN        NaN        NaN   

                                                        ...           mail  \
count  StateName7 StateName8 StateName9 StateName10     ...     StateMail9   
siteid                                                  ...                  
2             NaN        NaN        NaN         NaN     ...            NaN   
5             NaN        NaN        NaN         NaN     ...            NaN   
8             NaN        NaN        NaN         NaN     ...            NaN   
16            NaN        NaN        NaN         NaN     ...            NaN   
17            NaN        NaN        NaN         NaN     ...            NaN   
18            NaN        NaN        NaN         NaN     ...            NaN   
19            NaN        NaN        NaN         NaN     ...            NaN   
20            NaN        NaN        NaN         NaN     ...            NaN   
21            NaN        NaN        NaN         NaN     ...            NaN   
30            NaN        NaN        NaN         NaN     ...            NaN   
33            NaN        NaN        NaN         NaN     ...            NaN   
35            NaN        NaN        NaN         NaN     ...            NaN   
36            NaN        NaN        NaN         NaN     ...            NaN   

                                                                    \
count  StateMail10 StateMail11 StateMail12 StateMail13 StateMail14   
siteid                                                               
2              NaN         NaN         NaN         NaN         NaN   
5              NaN         NaN         NaN         NaN         NaN   
8              NaN         NaN         NaN         NaN         NaN   
16             NaN         NaN         NaN         NaN         NaN   
17             NaN         NaN         NaN         NaN         NaN   
18             NaN         NaN         NaN         NaN         NaN   
19             NaN         NaN         NaN         NaN         NaN   
20             NaN         NaN         NaN         NaN         NaN   
21             NaN         NaN         NaN         NaN         NaN   
30             NaN         NaN         NaN         NaN         NaN   
33             NaN         NaN         NaN         NaN         NaN   
35             NaN         NaN         NaN         NaN         NaN   
36             NaN         NaN         NaN         NaN         NaN   

                                                        
count  StateMail15 StateMail16 StateMail17 StateMail18  
siteid                                                  
2              NaN         NaN         NaN         NaN  
5              NaN         NaN         NaN         NaN  
8              NaN         NaN         NaN         NaN  
16             NaN         NaN         NaN         NaN  
17             NaN         

In [134]:
# Drop first row of index
corrected_varnames.columns = corrected_varnames.columns.droplevel(0)
corrected_varnames

count,StateName1,StateName2,StateName3,StateName4,StateName5,StateName6,StateName7,StateName8,StateName9,StateName10,...,StateMail9,StateMail10,StateMail11,StateMail12,StateMail13,StateMail14,StateMail15,StateMail16,StateMail17,StateMail18
siteid,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
corrected_varnames.reset_index()

count,siteid,StateName1,StateName2,StateName3,StateName4,StateName5,StateName6,StateName7,StateName8,StateName9,...,StateMail9,StateMail10,StateMail11,StateMail12,StateMail13,StateMail14,StateMail15,StateMail16,StateMail17,StateMail18
0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pd.unique(state_df.name.ravel())
# Ravel - Return the flattened underlying data as an ndarray